In [39]:
# Importing Libraries
import pandas as pd
import numpy as np
from keras.layers import Dense, GRU, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences


In [40]:
#Mount drive to access files in gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [41]:
# Read Train, Test and Gold dataset
df_train = pd.read_csv("/content/gdrive/MyDrive/NLP_Project/train.csv")
df_test = pd.read_csv("/content/gdrive/MyDrive/NLP_Project/dev.csv")
df_gold = pd.read_csv("/content/gdrive/MyDrive/NLP_Project/gold-test.csv")

In [42]:
# Function removing tags
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result

In [43]:
# Preprocessing data lowercase and removing hyperlinks
def preProcessData(data_frame):
    data_frame['text'] = (data_frame['text']).str.lower()
    data_frame['text'] = data_frame['text'].apply(lambda cw : remove_tags(cw))
    return data_frame

In [44]:
# Creating train, test and gold labels
X_train = preProcessData(df_train)['text']
X_test = preProcessData(df_test)['text']
X_gold = preProcessData(df_gold)['text']
Y_train = df_train['is_humor']
Y_test = df_test['is_humor']
Y_gold = df_gold['is_humor']

In [45]:
# Tokenizing the sentences and adding padding
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)

sequences_train = tokenizer.texts_to_sequences(X_train)
seq_matrix_train = pad_sequences(sequences_train, maxlen=200)

sequences_test = tokenizer.texts_to_sequences(X_test)
seq_matrix_test = pad_sequences(sequences_test, maxlen= 200)

sequences_gold = tokenizer.texts_to_sequences(X_gold)
seq_matrix_gold = pad_sequences(sequences_gold, maxlen= 200)

In [46]:
# Reading GloVe Embeddings
embeddings_index = dict()
f = open('/content/gdrive/MyDrive/NLP_Project/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))



Loaded 400000 word vectors.


In [47]:
# Creating weight matrix 
embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [48]:
# Metrics to evaluate model

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



In [49]:
%%time
# Initializing Model 
model = Sequential()
model.add(Embedding(vocabulary_size, 300, input_length=50, weights=[embedding_matrix], trainable=False))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy',f1_m, precision_m, recall_m])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 300)           6000000   
_________________________________________________________________
dropout_4 (Dropout)          (None, 50, 300)           0         
_________________________________________________________________
gru (GRU)                    (None, 100)               120600    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 6,120,701
Trainable params: 120,701
Non-trainable params: 6,000,000
_________________________________________________________________
CPU times: user 458 ms, sys: 41.5 ms, total: 500 ms
Wall time: 456 ms


In [50]:
%%time
# Training the model
model_glove.fit(seq_matrix_train, df_train['is_humor'], epochs = 10, batch_size= 128)

Epoch 1/10
63/63 [==============================] - 4s 26ms/step - loss: 0.1320 - binary_accuracy: 0.9475 - f1_m: 0.9572 - precision_m: 0.9574 - recall_m: 0.9578
Epoch 2/10
63/63 [==============================] - 1s 24ms/step - loss: 0.1071 - binary_accuracy: 0.9619 - f1_m: 0.9688 - precision_m: 0.9682 - recall_m: 0.9703
Epoch 3/10
63/63 [==============================] - 2s 24ms/step - loss: 0.0937 - binary_accuracy: 0.9663 - f1_m: 0.9726 - precision_m: 0.9700 - recall_m: 0.9757
Epoch 4/10
63/63 [==============================] - 2s 24ms/step - loss: 0.0852 - binary_accuracy: 0.9736 - f1_m: 0.9781 - precision_m: 0.9781 - recall_m: 0.9783
Epoch 5/10
63/63 [==============================] - 1s 24ms/step - loss: 0.0827 - binary_accuracy: 0.9709 - f1_m: 0.9755 - precision_m: 0.9783 - recall_m: 0.9735
Epoch 6/10
63/63 [==============================] - 2s 24ms/step - loss: 0.0657 - binary_accuracy: 0.9763 - f1_m: 0.9807 - precision_m: 0.9838 - recall_m: 0.9779
Epoch 7/10
63/63 [==========

In [51]:
# Evaluating model for gold and test data
gold_accuracy = model_glove.evaluate(seq_matrix_gold, Y_gold)
test_accuracy = model_glove.evaluate(seq_matrix_test, Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n F1 Score:{:0.3f}'.format(test_accuracy[0],test_accuracy[1]*100, test_accuracy[2]))
print('Gold set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n F1 Score:{:0.3f}'.format(gold_accuracy[0],gold_accuracy[1]*100, gold_accuracy[2]))

32/32 [==============================] - 0s 9ms/step - loss: 0.6253 - binary_accuracy: 0.8430 - f1_m: 0.8753 - precision_m: 0.8698 - recall_m: 0.8868
Test set
  Loss: 0.625
  Accuracy: 84.300
 F1 Score:0.875
Gold set
  Loss: 0.628
  Accuracy: 83.800
 F1 Score:0.867
